In [2]:
import os

In [3]:
root_dir = os.path.abspath('../')
data_dir = os.path.join(root_dir, 'data')
raw_data_dir = os.path.join(data_dir,'raw_data')
clean_data_dir = os.path.join(data_dir, 'clean_data')
embeddings_dir = os.path.join(data_dir, 'glove.twitter.27B')
embeddings_file = os.path.join(embeddings_dir, 'glove.twitter.27B.200d.txt')
print embeddings_file

/Users/ashutoshchoudhary/Ashutosh/UMass/Semester2/690N/Project/conversational_agent_personified/data/glove.twitter.27B/glove.twitter.27B.200d.txt


In [17]:
def construct_vocabulary():
    vocabulary = {}
    print "Start filtering word embeddings..."
    for sub in os.listdir(clean_data_dir):
        if sub.startswith("."): 
            continue  # ignore other files like .DS_Store
        else:
            sub_dir = os.path.join(clean_data_dir,sub)
            for file_name in os.listdir(sub_dir):
                if file_name.startswith("."):
                    continue  # ignore other files like .DS_Store
                else:
                    full_filename = os.path.join(sub_dir,file_name)
                    if vocabulary.has_key()

In [18]:
construct_vocabulary()

Start filtering word embeddings...
/Users/ashutoshchoudhary/Ashutosh/UMass/Semester2/690N/Project/conversational_agent_personified/data/clean_data/A/A Few Good Men.txt
/Users/ashutoshchoudhary/Ashutosh/UMass/Semester2/690N/Project/conversational_agent_personified/data/clean_data/A/A Most Violent Year.txt
/Users/ashutoshchoudhary/Ashutosh/UMass/Semester2/690N/Project/conversational_agent_personified/data/clean_data/A/Above the Law.txt
/Users/ashutoshchoudhary/Ashutosh/UMass/Semester2/690N/Project/conversational_agent_personified/data/clean_data/A/Absolute Power.txt
/Users/ashutoshchoudhary/Ashutosh/UMass/Semester2/690N/Project/conversational_agent_personified/data/clean_data/A/Abyss, The.txt
/Users/ashutoshchoudhary/Ashutosh/UMass/Semester2/690N/Project/conversational_agent_personified/data/clean_data/A/Adaptation.txt
/Users/ashutoshchoudhary/Ashutosh/UMass/Semester2/690N/Project/conversational_agent_personified/data/clean_data/A/Addams Family, The.txt
/Users/ashutoshchoudhary/Ashutosh/